In [1]:
!mkdir data/ 
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar -O data/ILSVRC2012_img_val.tar
!tar -xf data/ILSVRC2012_img_val.tar -C data/
!wget https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/imagenet_classes.txt
!wget https://raw.githubusercontent.com/yqihao/PTMValidations/main/ILSVRC2012_validation_ground_truth.txt

--2022-05-01 17:15:33--  https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
Resolving image-net.org (image-net.org)... 171.64.68.16
Connecting to image-net.org (image-net.org)|171.64.68.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6744924160 (6.3G) [application/x-tar]
Saving to: ‘data/ILSVRC2012_img_val.tar’

data/ILSVRC2012_img 100%[===================>]   6.28G  23.5MB/s    in 3m 23s  

2022-05-01 17:18:56 (31.7 MB/s) - ‘data/ILSVRC2012_img_val.tar’ saved [6744924160/6744924160]

--2022-05-01 17:19:27--  https://raw.githubusercontent.com/Evolving-AI-Lab/ppgn/master/misc/map_clsloc.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24366 (24K) [text/plain]
Saving to: ‘map_clsloc.txt’

map_clsloc.txt      100%[====

In [5]:
import tensorflow as tf
import numpy as np

model = tf.keras.applications.ResNet50(weights='imagenet')

mapping_file = open('map_clsloc.txt',mode='r')

validation_file = open('ILSVRC2012_validation_ground_truth.txt',mode='r')

validation_list = validation_file.readlines()
mapping_list_from = mapping_file.readlines()
mapping_list_to = []
for i in range(len(mapping_list_from)):
  mapping_list_to.append(mapping_list_from[i].split(" ")[1])
  mapping_list_from[i] = mapping_list_from[i].split(" ")[0]

for i in range(len(validation_list)):
  validation_list[i] = validation_list[i].split('\n')[0]

102981632/102967424 [==============================] - 1s 0us/step


In [6]:
correct = 0
top_5 = 0

it = 50000

for i in range(it):
  img_path = f'data/ILSVRC2012_val_{(i+1):08d}.JPEG'
  img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
  x = tf.keras.preprocessing.image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = tf.keras.applications.resnet.preprocess_input(x)
  preds = model.predict(x)

  id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=1)[0][0][0]
  index = mapping_list_from.index(id)

  if mapping_list_to[index] == validation_list[i]:
    correct = correct + 1
    top_5 = top_5 + 1
  else:
    for ii in range(1,5):
      id = tf.keras.applications.imagenet_utils.decode_predictions(preds, top=5)[0][ii][0]
      index = mapping_list_from.index(id)
      if mapping_list_to[index] == validation_list[i]:
        top_5 = top_5 + 1
        break

  if i!=0 and i%500==0:
    print(str(int(i/500))+'%, Top 1:' + str(correct/i) + ' Top 5:' + str(top_5/i))

accuracy = correct/it*100
top_5_accuracy = top_5/it*100
print('Top 1 Accuracy:'+str(accuracy)+'%')
print('Top 5 Accuracy:'+str(top_5_accuracy)+'%')

1%, Top 1:0.696 Top 5:0.896
2%, Top 1:0.695 Top 5:0.899
3%, Top 1:0.698 Top 5:0.8946666666666667
4%, Top 1:0.686 Top 5:0.887
5%, Top 1:0.6944 Top 5:0.8888
6%, Top 1:0.692 Top 5:0.8883333333333333
7%, Top 1:0.6888571428571428 Top 5:0.8917142857142857
8%, Top 1:0.68775 Top 5:0.89125
9%, Top 1:0.6871111111111111 Top 5:0.8893333333333333
10%, Top 1:0.687 Top 5:0.8886
11%, Top 1:0.6863636363636364 Top 5:0.8869090909090909
12%, Top 1:0.6858333333333333 Top 5:0.8868333333333334
13%, Top 1:0.6852307692307692 Top 5:0.8869230769230769
14%, Top 1:0.686 Top 5:0.887
15%, Top 1:0.6874666666666667 Top 5:0.8866666666666667
16%, Top 1:0.685875 Top 5:0.8865
17%, Top 1:0.6854117647058824 Top 5:0.8861176470588236
18%, Top 1:0.6843333333333333 Top 5:0.8845555555555555
19%, Top 1:0.6856842105263158 Top 5:0.8846315789473684
20%, Top 1:0.6851 Top 5:0.8835
21%, Top 1:0.684952380952381 Top 5:0.884
22%, Top 1:0.6854545454545454 Top 5:0.8831818181818182
23%, Top 1:0.6852173913043478 Top 5:0.8833913043478261
24%, 